<a href="https://colab.research.google.com/github/rajathpatel23/Chord-implementation/blob/master/Fine_tune_resnet_MT_STFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.applications.resnet import ResNet50
from sklearn.externals import joblib
import cv2

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model, load_model
from pickle import dump
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras.regularizers import l2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def get_tuning(X_in):
      tuned_out = []
      for x in range(len(X_in)):
            if X_in[x][0] >= 0.5:
                  tuned_out.append(1)
            else:
                  tuned_out.append(0)
      return tuned_out

def get_confusion_matrix(y_t, y_pred):
      tn, fp, fn, tp = confusion_matrix(y_t, y_pred).ravel()
      sensitivity = tp/(tp + fn)
      specificity = tn/(tn + fp)
      PPV = tp/(tp + fp)
      NPV = tn/(tn + fn)
      return sensitivity, specificity, PPV, NPV

def get_aucpr(y_t, y_pred):
      precision, recall, thres  = precision_recall_curve(y_t, y_pred)
      aucpr = auc(recall, precision)
      return aucpr


def get_processed_X(X):
      X_copy  = X
      for arr in range(len(X_copy)):
            a = np.min(X_copy[arr])
            b = np.max(X_copy[arr])
            X_copy[arr] = np.divide(np.subtract(X_copy[arr], a), (b-a))
      return X_copy 
#%%
def get_reshape(X):
      X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
      return X

In [0]:
train_dict_1 = joblib.load('/content/drive/My Drive/Dataset/train_stft_data_1.pkl')
train_dict_2 = joblib.load('/content/drive/My Drive/Dataset/train_stft_data_2.pkl')

In [0]:
X = np.concatenate((train_dict_1["stft"], train_dict_2["stft"]))
y = np.concatenate((train_dict_1["MT"], train_dict_2["MT"]))

In [0]:
del train_dict_1
del train_dict_2

In [0]:
X_new = [0 for x in range(len(X))]
dim = (224, 224)
i = 0
for elem in X:
  elem_new = cv2.resize(elem, dim, interpolation = cv2.INTER_AREA)
#   print(elem_new.shape)
  X_new[i] = (elem_new)  
  i+=1
  
del X
X_new = np.array(X_new)
X_new = get_processed_X(X_new)
# X = np.stack((X,)*3, axis=-1)
# print(X.shape)

In [0]:
X_new = np.stack((X_new,)*3, axis=-1)
print(X_new.shape)

(7683, 224, 224, 3)


In [0]:
model = load_model('/content/drive/My Drive/Dataset/Resnet_model_new/Resnet_freeze_09172019.h5')
a = 0
for layer in model.layers:
  if layer.trainable:
    print(True)
  else:
    layer.trainable = True
    a+=1
print(a)
a = 0
for layer in model.layers:
  if layer.trainable:
    a+=1
print(a)









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
True
176
177


In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam')
from sklearn.utils import class_weight
from keras.callbacks import ModelCheckpoint, EarlyStopping

sample_weights = class_weight.compute_sample_weight(class_weight="balanced", y=y)
callbacks = [EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True), 
             ModelCheckpoint(filepath='/content/drive/My Drive/Dataset/Resnet_model_new/Resnet_fine_tuned_09172019.h5', monitor='loss', save_best_only=True)]
history = model.fit(X_new, y,epochs=40,batch_size=64,shuffle=True, verbose=1, 
                    sample_weight=sample_weights, callbacks=callbacks)

Epoch 1/40
7683/7683 [==============================] - 213s 28ms/step - loss: 1.1019
Epoch 2/40
7683/7683 [==============================] - 198s 26ms/step - loss: 0.6814
Epoch 3/40
7683/7683 [==============================] - 197s 26ms/step - loss: 0.6112
Epoch 4/40
7683/7683 [==============================] - 197s 26ms/step - loss: 0.6000
Epoch 5/40
7683/7683 [==============================] - 197s 26ms/step - loss: 0.6348
Epoch 6/40
7683/7683 [==============================] - 197s 26ms/step - loss: 0.5994
Epoch 7/40
7683/7683 [==============================] - 197s 26ms/step - loss: 0.6523
Epoch 8/40
7683/7683 [==============================] - 197s 26ms/step - loss: 0.8004
Epoch 9/40
7683/7683 [==============================] - 196s 26ms/step - loss: 0.7934


In [5]:
model = load_model('/content/drive/My Drive/Dataset/Resnet_model_new/Resnet_fine_tuned_09172019.h5')
from keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, cohen_kappa_score, auc, roc_curve
from sklearn.metrics import average_precision_score, confusion_matrix, precision_recall_curve, classification_report
def performance_analysis(y_2, y_test_1):      
      tuned_y_2 = get_tuning(y_2)
      roc_auc_test = roc_auc_score(y_test_1, y_2)
      print("The test roc is : ", roc_auc_test)
      aucpr_test = get_aucpr(y_test_1, y_2)
      print("The test aucpr is: ", aucpr_test)
      sense_test_, specs_test_, PPV_test_, NPV_test_ = get_confusion_matrix(y_test_1, tuned_y_2)
      print("The test sensitivity is: ", sense_test_)
      print("The test specificity is: ", specs_test_)
      print("The test PPV is: ", PPV_test_)
      print("The test NPV is: ", NPV_test_)
      return True









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
test_1 = joblib.load('/content/drive/My Drive/Dataset/test_stft_data_1.pkl')
test_2 = joblib.load('/content/drive/My Drive/Dataset/test_stft_data_2.pkl')
X = np.concatenate((test_1["stft"], test_2["stft"]))
y_test = np.concatenate((test_1["MT"], test_2["MT"]))
del test_1
del test_2

In [7]:
X_new = [0 for x in range(len(X))]
dim = (224, 224)
i = 0
for elem in X:
  elem_new = cv2.resize(elem, dim, interpolation = cv2.INTER_AREA)
#   print(elem_new.shape)
  X_new[i] = (elem_new)  
  i+=1

del X
X_new = np.array(X_new)
X_new = get_processed_X(X_new)
X_new = np.stack((X_new,)*3, axis=-1)
print(X_new.shape)

(6831, 224, 224, 3)


In [8]:
y_pred_MT = model.predict(X_new)
performance_analysis(y_pred_MT, y_test)

The test roc is :  0.8432858686449396
The test aucpr is:  0.13090330702849626
The test sensitivity is:  0.07971014492753623
The test specificity is:  0.9958165247273271
The test PPV is:  0.28205128205128205
The test NPV is:  0.981301531213192


True